In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = ChatGroq(
    temperature=0,
    model="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-i-itc/job/R-51569")
page_data = loader.load().pop().page_content
print(page_data)





















Software Engineer I , ITC











































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu





In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I, ITC',
 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role',
 'skills': ['Front-End Skills in React or similar technologies',
  'Server-side programming languages such as Node.js, Python or similar',
  'NoSQL databases',
  'API Development',
  'Cloud Technologies (AWS)',
  'Version Control (Git, GitHub)',
  'Testing Frameworks (unit testing, integration testing, TDD)',
  'Problem-Solving',
  'Collaboration'],
 'description': 'Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Front-End Development using modern JavaScript frameworks, Back-End Development using Node.js, Python or similar, Database Management, Testing & Quality Assurance, Performance Optimization, CI/CD Integration'}

In [7]:
type(json_res)

dict

In [11]:
job = json_res
job

{'role': 'Software Engineer I, ITC',
 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role',
 'skills': ['Front-End Skills in React or similar technologies',
  'Server-side programming languages such as Node.js, Python or similar',
  'NoSQL databases',
  'API Development',
  'Cloud Technologies (AWS)',
  'Version Control (Git, GitHub)',
  'Testing Frameworks (unit testing, integration testing, TDD)',
  'Problem-Solving',
  'Collaboration'],
 'description': 'Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Front-End Development using modern JavaScript frameworks, Back-End Development using Node.js, Python or similar, Database Management, Testing & Quality Assurance, Performance Optimization, CI/CD Integration'}

In [14]:
job['skills']

['Front-End Skills in React or similar technologies',
 'Server-side programming languages such as Node.js, Python or similar',
 'NoSQL databases',
 'API Development',
 'Cloud Technologies (AWS)',
 'Version Control (Git, GitHub)',
 'Testing Frameworks (unit testing, integration testing, TDD)',
 'Problem-Solving',
 'Collaboration']

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Your Business Needs

Dear Hiring Manager,

I came across your job posting for a Software Engineer I, ITC, and I'm excited to introduce AtliQ, a leading AI & Software Consulting company that can fulfill your requirements. With our expertise in designing and developing scalable web applications and services, we're confident that we can meet your needs.

Our team of skilled engineers has extensive experience in Front-End Development using modern JavaScript frameworks like React, as well as Back-End Development using Node.js, Python, and similar technologies. We're well-versed in Database Management, NoSQL databases, API Development, Cloud Technologies (AWS), and Version Control (Git, GitHub). Our testing capabilities include unit testing, integration testing, and TDD, ensuring the highest quality of our deliverables.

At AtliQ, we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost r